<a href="https://colab.research.google.com/github/SakayanagiTOYOTA/forLLM_edu/blob/main/BuildaSimpleLLMApplicationwithLCEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

引用元：
* [LangChain Tutorials](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/)
* [VSCodeでJupyterを使う](https://python-work.com/vscode-ext-jupyter/)

環境変数設定：

In [ ]:
# AzureOpenAIの場合
import os
os.environ["OPENAI_API_VERSION"] = "2024-02-15-preview"
os.environ["AZURE_OPENAI_API_KEY"] = "xxxxxxxx"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://aoai-aoai04-japaneast.openai.azure.com/'
AZURE_DEPLOYMENT='gpt-4o-2024-05-13'

In [ ]:
# AWS Bedrockの場合
import os
os.environ['AWS_ACCESS_KEY_ID'] = 'xxxxx'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'xxxxxx'
os.environ['AWS_SESSION_TOKEN'] = 'xxxxxx'
os.environ['AWS_DEFAULT_REGION'] = 'ap-northeast-1'

# Build Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After reading this tutorial, you'll have a high level overview of:

* Using [language models](https://python.langchain.com/v0.2/docs/concepts/#chat-models)

* Using [PromptTemplates](https://python.langchain.com/v0.2/docs/concepts/#prompt-templates) and [OutputParsers](https://python.langchain.com/v0.2/docs/concepts/#output-parsers)

* Using [LangChain Expression Language (LCEL)](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel) to chain components together

* Deploying your application with [LangServe](https://python.langchain.com/v0.2/docs/concepts/#langserve)

Let's dive in!

# Using Language Models
First up, let's learn how to use a language model by itself. LangChain supports many different language models that you can use interchangeably - select the one you want to use below!

In [ ]:
#AzureOpenAIの場合
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(azure_deployment=AZURE_DEPLOYMENT)

In [ ]:
#AWS Bedrockの場合
from langchain_aws.chat_models import ChatBedrock
model = ChatBedrock(model_id='anthropic.claude-3-haiku-20240307-v1:0')

Let's first use the model directly. ChatModels are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the .invoke method.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

API Reference:[HumanMessage](https://python.langchain.com/v0.2/api_reference/core/messages/langchain_core.messages.human.HumanMessage.html) | [SystemMessage](https://python.langchain.com/v0.2/api_reference/core/messages/langchain_core.messages.system.SystemMessage.html)

# OutputParsers

Notice that the response from the model is an AIMessage. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

We first import the simple output parser.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

API Reference:[StrOutputParser](https://python.langchain.com/v0.2/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html)

One way to use it is to use it by itself. For example, we could save the result of the language model call and then pass it to the parser.

In [ ]:
result = model.invoke(messages)
parser.invoke(result)

More commonly, we can "chain" the model with this output parser. This means this output parser will get called every time in this chain. This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).

We can easily create the chain using the | operator. The | operator is used in LangChain to combine two elements together.

In [ ]:
chain = model | parser

chain.invoke(messages)

# Prompt Templates

Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

PromptTemplates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

Let's create a PromptTemplate here. It will take in two user variables:

* language: The language to translate text into
* text: The text to translate

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

API Reference:[ChatPromptTemplate](https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html)

First, let's create a string that we will format to be the system message:

In [ ]:
system_template = "Translate the following into {language}:"

Next, we can create the PromptTemplate. This will be a combination of the system_template as well as a simpler template for where to put the text to be translated

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself

In [ ]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

We can see that it returns a ChatPromptValue that consists of two messages. If we want to access the messages directly we do:

In [ ]:
result.to_messages()

# Chaining together components with LCEL

We can now combine this with the model and the output parser from above using the pipe (|) operator:

In [ ]:
chain = prompt_template | model | parser

chain.invoke({"language": "italian", "text": "hi"})

This is a simple example of using [LangChain Expression Language (LCEL)](https://python.langchain.com/v0.2/docs/concepts/#langchain-expression-language-lcel) to chain together LangChain modules. There are several benefits to this approach, including optimized streaming and tracing support.

# Serving with LangServe
Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

## Server
To create a server for our application we'll make a serve.py file. This will contain our logic for serving our application. It consists of three things:

1. The definition of our chain that we just built above
2. Our FastAPI app
3. A definition of a route from which to serve the chain, which is done with langserve.add_routes

serve.py:
```python
#!/usr/bin/env python
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import AzureChatOpenAI
from langserve import add_routes

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
model = AzureChatOpenAI(azure_deployment=AZURE_DEPLOYMENT)

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route
add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)
```

API Reference:[ChatPromptTemplate](https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html) | [StrOutputParser](https://python.langchain.com/v0.2/api_reference/core/output_parsers/langchain_core.output_parsers.string.StrOutputParser.html) | [ChatOpenAI](https://python.langchain.com/v0.2/api_reference/openai/chat_models/langchain_openai.chat_models.base.ChatOpenAI.html)

And that's it! If we execute this file:

```
python serve.py
```
we should see our chain being served at http://localhost:8000.

## Playground

Every LangServe service comes with a simple [built-in](https://github.com/langchain-ai/langserve/blob/main/README.md#playground) UI for configuring and invoking the application with streaming output and visibility into intermediate steps. Head to [http://localhost:8000/chain/playground/](http://localhost:8000/chain/playground/) to try it out! Pass in the same inputs as before - {"language": "italian", "text": "hi"} - and it should respond same as before.

## Client

Now let's set up a client for programmatically interacting with our service. We can easily do this with the [langserve.RemoteRunnable](https://python.langchain.com/v0.2/docs/langserve/#client). Using this, we can interact with the served chain as if it were running client-side.

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

To learn more about the many other features of LangServe [head here](https://python.langchain.com/v0.2/docs/langserve/).

# Conclusion

That's it! In this tutorial you've learned how to create your first simple LLM application. You've learned how to work with language models, how to parse their outputs, how to create a prompt template, chaining them with LCEL, how to get great observability into chains you create with LangSmith, and how to deploy them with LangServe.

This just scratches the surface of what you will want to learn to become a proficient AI Engineer. Luckily - we've got a lot of other resources!

For further reading on the core concepts of LangChain, we've got detailed [Conceptual Guides](https://python.langchain.com/v0.2/docs/concepts/).

If you have more specific questions on these concepts, check out the following sections of the how-to guides:

* [LangChain Expression Language (LCEL)](https://python.langchain.com/v0.2/docs/how_to/#langchain-expression-language-lcel)
* [Prompt templates](https://python.langchain.com/v0.2/docs/how_to/#prompt-templates)
* [Chat models](https://python.langchain.com/v0.2/docs/how_to/#chat-models)
* [Output parsers](https://python.langchain.com/v0.2/docs/how_to/#output-parsers)
* [LangServe](https://python.langchain.com/v0.2/docs/langserve/)